# Modules

In [1]:
%matplotlib widget
import os
import glob
import psfex
import pickle
import sncosmo
import numpy as np
import pandas as pd
import iminuit as im
import astropy as ap
import ipywidgets as ipw
import matplotlib.pyplot as plt
import ztfquery
import ztfimg
import ztflc

from ztfimg import dao
from ztfimg import image
from ztflc import fitter
from ztflc import diffdata
from astropy.io import fits
from ztfquery import marshal
from astrobject import photometry
from ztflc import forcephotometry
from scipy.stats import multivariate_normal
from sncosmo.models import Source, Model, get_source
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sncosmo import get_bandpass, get_magsystem
from sncosmo.photdata import photometric_data

### Open relevant data, create stars list from chosen catalog, and choose one to fit

In [12]:
root = "../Data/"
psfoutcat = root + 'config_ZOGY/BlackGEM_default/psfex_out.cat'
mask = root + "ztf_20190124095417_000403_zr_c01_o_q1_mskimg.fits"
psfcat = root + "ztf_20190124095417_000403_zr_c01_o_q1_psfcat.fits"
sciimg = root + "ztf_20190124095417_000403_zr_c01_o_q1_sciimg.fits"
psfcent = root + "ztf_20190124095417_000403_zr_c01_o_q1_sciimgdaopsfcent.fits"
diffimgpath = root + 'ztf_20190124095417_000403_zr_c01_o_q1_scimrefdiffimg.fits.fz'
diffpsfpath = root + 'ztf_20190124095417_000403_zr_c01_o_q1_diffimgpsf.fits'

sdul = fits.open(sciimg)
pdul = fits.open(psfcat)
cdul = fits.open(psfcent)

cdata = cdul[0].data
header = pdul[0].header

#df_psfcat = pd.DataFrame(pdul[1].data)
#x_list, y_list = df_psfcat['xpos'], df_psfcat['ypos']

df_outcat = pd.read_csv(psfoutcat, delim_whitespace=True)
x_list, y_list = df_outcat[['x', 'y']].values.T
xy_list = df_outcat[['x', 'y']].values

zimg = photometry.Image(sciimg, background=0)
ra_list, dec_list = np.asarray([zimg.pixel_to_coords(x, y) for x, y in xy_list]).T

z = image.ScienceImage(sciimg, mask)
z.load_source_background()
z.set_catalog(df_outcat, 'outcat')

ind = 500
row, col = x_list[ind], y_list[ind]

plt.close()
ax = z.show('dataclean')
ax.scatter(x_list, y_list, color='C1', marker='+')

### PSF choice

In [13]:
subdir_list = [subdir_name for subdir_name in os.listdir('../Data/config_ZOGY/')]

psfchoice = ipw.Dropdown(options=subdir_list,
                         description='Which psfex.config?',
                         value=subdir_list[1])
def choose_psf(psfdir):
    global psfimg, psfimg_cent, psfimg_smpl, psfobj
    psfilepath = "../Data/config_ZOGY/" + psfdir + '/test.psf'
    psfobj = psfex.PSFEx(psfilepath)
    psfimg = psfobj.get_rec(row, col)

int_choose_psf = ipw.interactive(choose_psf, psfdir=psfchoice)
display(int_choose_psf)

interactive(children=(Dropdown(description='Which psfex.config?', index=1, options=('psfex_wrapper', 'BlackGEM…

### Create `.fits` file from the chosen `.psf` file

In [14]:
FILE_OUT = sciimg[:-11] + 'psfeximg.fits'

psfeximg = fits.HDUList(fits.PrimaryHDU(data=psfimg,
                                        header=header))
psfeximg.writeto(FILE_OUT, overwrite=True)

### Assign the data to `ztflc`

In [15]:
diffpsfex = '../Data/ztf_20190124095417_000403_zr_c01_o_q1_psfeximg.fits'
fit_obj = ztflc.diffdata.DiffData(diffimgpath,
                                  #diffpsfpath,
                                  diffpsfex,
                                  (ra_list[ind],
                                   dec_list[ind]))

### Fit result

In [16]:
fit_obj.fit_flux()

{'sigma': 7.457749773544009,
 'sigma.err': 0.2293083722802518,
 'ampl': -68.18377927782038,
 'ampl.err': 30.443187019151466,
 'fval': 3626.936199731239,
 'chi2': 527.9565889112422,
 'chi2dof': 1.0018151592243685}

## Compare the used psf

### Assign `diffpsfimg` to a `DiffData` object to have the correct psf slice

In [17]:
comp_obj = ztflc.diffdata.DiffData(diffimgpath,
                                   diffpsfpath,
                                   (ra_list[ind],
                                    dec_list[ind]))
pdata = comp_obj.psfimg

### Norm one on the other

In [18]:
psfex_max = np.max(np.ravel(psfimg))

pdata_norm = np.sum(np.ravel(pdata))
pdata_normed = pdata/pdata_norm
pdata_normed_max = np.max(pdata_normed)
pdata_scaled = pdata_normed*psfex_max/pdata_normed_max

### Make a patch to compute the difference

In [19]:
xx, yy = np.shape(pdata)
x, y = xx/2, yy/2
dx, dy = np.shape(psfimg)

x_slice = slice(int(x-dx/2+0.5), int(x+dx/2+0.5))
y_slice = slice(int(y-dy/2+0.5), int(y+dy/2+0.5))
pdata_patch = pdata_scaled[y_slice].T[x_slice].T

### Comparison plot

In [35]:
plt.close()
fig = plt.figure(figsize=[5, 8])

ax = fig.add_subplot(321)
ax.imshow(comp_obj.diffimg[y_slice].T[x_slice].T)
plt.title('diffimg')

ax = fig.add_subplot(322)
ax.imshow(pdata_patch)
plt.title('diffimgpsf.fits')

ax = fig.add_subplot(323)
ax.imshow(comp_obj.diffimg[y_slice].T[x_slice].T)
plt.title('diffimg')

ax = fig.add_subplot(324)
ax.imshow(psfimg)
plt.title('test.psf')

ax = fig.add_subplot(326)
im = ax.imshow((pdata_patch-psfimg)/np.max(psfimg))
plt.title('Diff/max_middle')

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

fig.suptitle('PSF diff for ' + psfchoice.value, fontsize=11)

fig.savefig('../Images/' + 'png' + '/diffimgpsf_testpsf_diff_' + psfchoice.value + '.png',
            bbox_inches='tight')
fig.savefig('../Images/' + 'pdf' + '/diffimgpsf_testpsf_diff_' + psfchoice.value + '.pdf',
            bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …